In [6]:
from pedect.converter.ConverterToImages import *
from pedect.config.BasicConfig import BasicConfig
from pedect.utils.detector.debug import debug
from pedect.utils.detector.train import train
from pedect.utils.detector.evaluate import evaluate
import os


In [7]:
# convertVideosToImages([("caltech" , "set01", "V000"), ("caltech", "set02", "V000")])

In [8]:
config = BasicConfig()
# debug(config)

In [9]:
# train(config)

In [11]:
# evaluate(config)

In [1]:
from pedect.utils.constants import *
from pedect.utils.metrics import *
from pedect.dataset.CaltechDataset import CaltechDataset
from pedect.input.inputProcessing import *
from pedect.utils.IdGenerator import IdGenerator
from pedect.predictor.FakePredictor import FakePredictor
import cv2
from tracker import re3_tracker 

chosenDataset = CaltechDataset(CALTECH_DIR)
predictor = FakePredictor(chosenDataset, "set01", "V000")
tracker = re3_tracker.Re3Tracker()


Format mjpeg detected only with low score of 25, misdetection possible!


Restoring:
re3/conv1/W_conv -> 	[11, 11, 3, 96] = 0MB
re3/conv1/b_conv -> 	[96] = 0MB
re3/conv1_skip/W_conv -> 	[1, 1, 96, 16] = 0MB
re3/conv1_skip/b_conv -> 	[16] = 0MB
re3/conv1_skip/prelu -> 	[16] = 0MB
re3/conv2/W_conv -> 	[5, 5, 48, 256] = 1MB
re3/conv2/b_conv -> 	[256] = 0MB
re3/conv2_skip/W_conv -> 	[1, 1, 256, 32] = 0MB
re3/conv2_skip/b_conv -> 	[32] = 0MB
re3/conv2_skip/prelu -> 	[32] = 0MB
re3/conv3/W_conv -> 	[3, 3, 256, 384] = 3MB
re3/conv3/b_conv -> 	[384] = 0MB
re3/conv4/W_conv -> 	[3, 3, 192, 384] = 2MB
re3/conv4/b_conv -> 	[384] = 0MB
re3/conv5/W_conv -> 	[3, 3, 192, 256] = 1MB
re3/conv5/b_conv -> 	[256] = 0MB
re3/conv5_skip/W_conv -> 	[1, 1, 256, 64] = 0MB
re3/conv5_skip/b_conv -> 	[64] = 0MB
re3/conv5_skip/prelu -> 	[64] = 0MB
re3/fc6/W_fc -> 	[74208, 1024] = 303MB
re3/fc6/b_fc -> 	[1024] = 0MB
re3/fc_output/W_fc -> 	[512, 4] = 0MB
re3/fc_output/b_fc -> 	[4] = 0MB
re3/lstm1/rnn/lstm_cell/bias -> 	[2048] = 0MB
re3/lstm1/rnn/lstm_cell/kernel -> 	[1536, 2048] = 12MB
re3/

In [9]:
import random
from pedect.utils.trackedObjectsOperations import *
from pedect.predictor.PredictedBox import *
from pedect.config.BasicConfig import BasicConfig
class MyConfig(BasicConfig):
    createThreshold = 0.90
    removeThreshold = 0.5
    surviveThreshold = 0.2
    surviveMovePercent = 0.0
    maxAge = 100
config = MyConfig()

def addRectanglesToImage(image, activeObjects, predictedBBoxes, createThreshold):
    for predBox in predictedBBoxes:
        if predBox.getProb() >= createThreshold:
            color = [0, 255, 0]
            box = predBox.getPos()
            cv2.rectangle(image, (box[0]-1, box[1]-1), (box[2]+1, box[3]+1), color, 2)
    
    for activeId, v in activeObjects.items():
        bbox = v.getPos()
        tracker.track(activeId, image, bbox)
        cv2.rectangle(image, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), [0, 0, 255], 2)

def playVideo(predictor, tracker, config):
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', 640, 480)
    activeObjects = {}
    video = predictor.getVideo()
    for frameNr in range(min(predictor.getLength(), 100)):
        image = video[frameNr] #predictor.getFrame(frameNr)
        activeObjects = refreshTrackedObjects(tracker, image, activeObjects)
        predictedBBoxes = predictor.predictForFrame(frameNr)
        activeObjects = moveOrDestroyTrackedObjects(activeObjects, predictedBBoxes, config.surviveMovePercent, config.surviveThreshold)
        activeObjects = createAndDestroyTrackedObjects(tracker, image, activeObjects, predictedBBoxes, config.createThreshold, config.removeThreshold, frameNr)
        addRectanglesToImage(image, activeObjects, predictedBBoxes, config.createThreshold)

        activeObjects = removeOldObjects(activeObjects, frameNr, config.maxAge)
        cv2.imshow('Video', image)
        cv2.waitKey(1)
    cv2.destroyAllWindows()
    
def evaluate(predictor, tracker, config, evalFunction):
    activeObjects = {}
    video = predictor.getVideo()
    results = []
    for frameNr in range(min(predictor.getLength(), 100)):
        image = video[frameNr] #predictor.getFrame(frameNr)
        activeObjects = refreshTrackedObjects(tracker, image, activeObjects)
        predictedBBoxes = predictor.predictForFrame(frameNr)
        activeObjects = moveOrDestroyTrackedObjects(activeObjects, predictedBBoxes, config.surviveMovePercent, config.surviveThreshold)
        activeObjects = createAndDestroyTrackedObjects(tracker, image, activeObjects, predictedBBoxes, config.createThreshold, config.removeThreshold, frameNr)
        addRectanglesToImage(image, activeObjects, predictedBBoxes, config.createThreshold)
        activeObjects = removeOldObjects(activeObjects, frameNr, config.maxAge)
        groundTruthObjects = predictor.getGroundTruthForFrame(frameNr)
        objects = [PredictedBox(v.getPos()[0], v.getPos()[1], v.getPos()[2], v.getPos()[3], v.getLabel(), 1.0)
                   for k, v in activeObjects.items()]
        results.append(evalFunction(objects, groundTruthObjects))
    return results

def randomEvalFunction(predictedObjects, actualObjects):
    s = 0
    for pObj in predictedObjects:
        for aObj in actualObjects:
            s += 1 if IOU(pObj.getPos(), aObj.getPos()) >= 0.5 else 0
    return s

def findRes(predictedObjects, actualObjects, acceptThreshold):
    taken = [False for _ in range(len(actualObjects))]
    results = []
    for pObj in predictedObjects:
        best = -1
        bestPos = -1
        for i in range(len(actualObjects)):
            iou = IOU(pObj.getBox(), actualObjects[i].getBox())
            if iou > best:
                best = iou
                bestPos = i
        if taken[bestPos] or best < acceptThreshold:
            results.append(False)
        else:
            results.append(True)
            taken[bestPos] = True
    return results
def mAP(predictedObjects, actualObjects):
    acceptThreshold = 0.5
    results = findRes(predictedObjects, actualObjects, acceptThreshold)
    results = [1 if a else 0 for a in results]
    return sum(results) / len(results)


playVideo(predictor, tracker, config)
# results = evaluate(predictor, tracker, config, mAP)

deprecated pixel format used, make sure you did set range correctly
 (repeated 5 more times)
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
dep

Current tracking speed:   21.338 FPS
Current image read speed: 100000.000 FPS
Mean tracking speed:      22.673 FPS



deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used

Current tracking speed:   20.860 FPS
Current image read speed: 100000.000 FPS
Mean tracking speed:      22.763 FPS



deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used

Current tracking speed:   22.756 FPS
Current image read speed: 100000.000 FPS
Mean tracking speed:      22.802 FPS



deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used

Current tracking speed:   21.322 FPS
Current image read speed: 100000.000 FPS
Mean tracking speed:      22.856 FPS



deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
Format mjpeg detected only with low score of 25, misdetection possible!
deprecated pixel format used

In [6]:
# from https://github.com/Cartucho/mAP
import glob
import json
import os
import shutil
import operator
import sys
import argparse
import math

import numpy as np
def log_average_miss_rate(precision, fp_cumsum, num_images):
    """
        log-average miss rate:
            Calculated by averaging miss rates at 9 evenly spaced FPPI points
            between 10e-2 and 10e0, in log-space.

        output:
                lamr | log-average miss rate
                mr | miss rate
                fppi | false positives per image

        references:
            [1] Dollar, Piotr, et al. "Pedestrian Detection: An Evaluation of the
               State of the Art." Pattern Analysis and Machine Intelligence, IEEE
               Transactions on 34.4 (2012): 743 - 761.
    """

    # if there were no detections of that class
    if precision.size == 0:
        lamr = 0
        mr = 1
        fppi = 0
        return lamr, mr, fppi

    fppi = fp_cumsum / float(num_images)
    mr = (1 - precision)

    fppi_tmp = np.insert(fppi, 0, -1.0)
    mr_tmp = np.insert(mr, 0, 1.0)

    # Use 9 evenly spaced reference points in log-space
    ref = np.logspace(-2.0, 0.0, num = 9)
    for i, ref_i in enumerate(ref):
        # np.where() will always find at least 1 index, since min(ref) = 0.01 and min(fppi_tmp) = -1.0
        j = np.where(fppi_tmp <= ref_i)[-1][-1]
        ref[i] = mr_tmp[j]

    # log(0) is undefined, so we use the np.maximum(1e-10, ref)
    lamr = math.exp(np.mean(np.log(np.maximum(1e-10, ref))))

    return lamr, mr, fppi

"""
 throw error and exit
"""
def error(msg):
    print(msg)
    sys.exit(0)

"""
 check if the number is a float between 0.0 and 1.0
"""
def is_float_between_0_and_1(value):
    try:
        val = float(value)
        if val > 0.0 and val < 1.0:
            return True
        else:
            return False
    except ValueError:
        return False

"""
 Calculate the AP given the recall and precision array
    1st) We compute a version of the measured precision/recall curve with
         precision monotonically decreasing
    2nd) We compute the AP as the area under this curve by numerical integration.
"""
def voc_ap(rec, prec):
    """
    --- Official matlab code VOC2012---
    mrec=[0 ; rec ; 1];
    mpre=[0 ; prec ; 0];
    for i=numel(mpre)-1:-1:1
            mpre(i)=max(mpre(i),mpre(i+1));
    end
    i=find(mrec(2:end)~=mrec(1:end-1))+1;
    ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    """
    rec.insert(0, 0.0) # insert 0.0 at begining of list
    rec.append(1.0) # insert 1.0 at end of list
    mrec = rec[:]
    prec.insert(0, 0.0) # insert 0.0 at begining of list
    prec.append(0.0) # insert 0.0 at end of list
    mpre = prec[:]
    """
     This part makes the precision monotonically decreasing
        (goes from the end to the beginning)
        matlab: for i=numel(mpre)-1:-1:1
                    mpre(i)=max(mpre(i),mpre(i+1));
    """
    # matlab indexes start in 1 but python in 0, so I have to do:
    #     range(start=(len(mpre) - 2), end=0, step=-1)
    # also the python function range excludes the end, resulting in:
    #     range(start=(len(mpre) - 2), end=-1, step=-1)
    for i in range(len(mpre)-2, -1, -1):
        mpre[i] = max(mpre[i], mpre[i+1])
    """
     This part creates a list of indexes where the recall changes
        matlab: i=find(mrec(2:end)~=mrec(1:end-1))+1;
    """
    i_list = []
    for i in range(1, len(mrec)):
        if mrec[i] != mrec[i-1]:
            i_list.append(i) # if it was matlab would be i + 1
    """
     The Average Precision (AP) is the area under the curve
        (numerical integration)
        matlab: ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    """
    ap = 0.0
    for i in i_list:
        ap += ((mrec[i]-mrec[i-1])*mpre[i])
    return ap, mrec, mpre


"""
 Convert the lines of a file to a list
"""
def file_lines_to_list(path):
    # open txt file lines to a list
    with open(path) as f:
        content = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

"""
 Draws text in image
"""
def draw_text_in_image(img, text, pos, color, line_width):
    font = cv2.FONT_HERSHEY_PLAIN
    fontScale = 1
    lineType = 1
    bottomLeftCornerOfText = pos
    cv2.putText(img, text,
            bottomLeftCornerOfText,
            font,
            fontScale,
            color,
            lineType)
    text_width, _ = cv2.getTextSize(text, font, fontScale, lineType)[0]
    return img, (line_width + text_width)

"""
 Plot - adjust axes
"""
def adjust_axes(r, t, fig, axes):
    # get text width for re-scaling
    bb = t.get_window_extent(renderer=r)
    text_width_inches = bb.width / fig.dpi
    # get axis width in inches
    current_fig_width = fig.get_figwidth()
    new_fig_width = current_fig_width + text_width_inches
    propotion = new_fig_width / current_fig_width
    # get axis limit
    x_lim = axes.get_xlim()
    axes.set_xlim([x_lim[0], x_lim[1]*propotion])

"""
 Draw plot using Matplotlib
"""
def draw_plot_func(dictionary, n_classes, window_title, plot_title, x_label, output_path, to_show, plot_color, true_p_bar):
    # sort the dictionary by decreasing value, into a list of tuples
    sorted_dic_by_value = sorted(dictionary.items(), key=operator.itemgetter(1))
    # unpacking the list of tuples into two lists
    sorted_keys, sorted_values = zip(*sorted_dic_by_value)
    # 
    if true_p_bar != "":
        """
         Special case to draw in:
            - green -> TP: True Positives (object detected and matches ground-truth)
            - red -> FP: False Positives (object detected but does not match ground-truth)
            - orange -> FN: False Negatives (object not detected but present in the ground-truth)
        """
        fp_sorted = []
        tp_sorted = []
        for key in sorted_keys:
            fp_sorted.append(dictionary[key] - true_p_bar[key])
            tp_sorted.append(true_p_bar[key])
        plt.barh(range(n_classes), fp_sorted, align='center', color='crimson', label='False Predictions')
        plt.barh(range(n_classes), tp_sorted, align='center', color='forestgreen', label='True Predictions', left=fp_sorted)
        # add legend
        plt.legend(loc='lower right')
        """
         Write number on side of bar
        """
        fig = plt.gcf() # gcf - get current figure
        axes = plt.gca()
        r = fig.canvas.get_renderer()
        for i, val in enumerate(sorted_values):
            fp_val = fp_sorted[i]
            tp_val = tp_sorted[i]
            fp_str_val = " " + str(fp_val)
            tp_str_val = fp_str_val + " " + str(tp_val)
            # trick to paint multicolor with offset:
            # first paint everything and then repaint the first number
            t = plt.text(val, i, tp_str_val, color='forestgreen', va='center', fontweight='bold')
            plt.text(val, i, fp_str_val, color='crimson', va='center', fontweight='bold')
            if i == (len(sorted_values)-1): # largest bar
                adjust_axes(r, t, fig, axes)
    else:
        plt.barh(range(n_classes), sorted_values, color=plot_color)
        """
         Write number on side of bar
        """
        fig = plt.gcf() # gcf - get current figure
        axes = plt.gca()
        r = fig.canvas.get_renderer()
        for i, val in enumerate(sorted_values):
            str_val = " " + str(val) # add a space before
            if val < 1.0:
                str_val = " {0:.2f}".format(val)
            t = plt.text(val, i, str_val, color=plot_color, va='center', fontweight='bold')
            # re-set axes to show number inside the figure
            if i == (len(sorted_values)-1): # largest bar
                adjust_axes(r, t, fig, axes)
    # set window title
    fig.canvas.set_window_title(window_title)
    # write classes in y axis
    tick_font_size = 12
    plt.yticks(range(n_classes), sorted_keys, fontsize=tick_font_size)
    """
     Re-scale height accordingly
    """
    init_height = fig.get_figheight()
    # comput the matrix height in points and inches
    dpi = fig.dpi
    height_pt = n_classes * (tick_font_size * 1.4) # 1.4 (some spacing)
    height_in = height_pt / dpi
    # compute the required figure height 
    top_margin = 0.15 # in percentage of the figure height
    bottom_margin = 0.05 # in percentage of the figure height
    figure_height = height_in / (1 - top_margin - bottom_margin)
    # set new height
    if figure_height > init_height:
        fig.set_figheight(figure_height)

    # set plot title
    plt.title(plot_title, fontsize=14)
    # set axis titles
    # plt.xlabel('classes')
    plt.xlabel(x_label, fontsize='large')
    # adjust size of window
    fig.tight_layout()
    # save the plot
    fig.savefig(output_path)
    # show image
    if to_show:
        plt.show()
    # close the plot
    plt.close()

In [10]:
from pedect.predictor.PredictedBox import PredictedBox
from pedect.utils.metrics import *
MINOVERLAP = 0.5
predicted = [PredictedBox(1, 2, 100, 200, 'person', 0.6), 
             PredictedBox(100, 40, 200, 120, 'people', 0.8), 
             PredictedBox(1, 2, 100, 200, 'person', 0.3)]

groundTruth = [PredictedBox(1, 2, 100, 200, 'person', 1.0), 
              PredictedBox(100, 40, 200, 120, 'people', 1.0), 
              PredictedBox(1, 2, 100, 200, 'person', 1.0)]
# remember to sort!!!!!!!!!!!!
classes = {p.getLabel(): ([], []) for p in predicted + groundTruth}
for obj in predicted:
    classes[obj.getLabel()][0].append(obj)
for obj in groundTruth:
    classes[obj.getLabel()][1].append(obj)

sum_AP = 0.0
ap_dictionary = {}
lamr_dictionary = {}
# open file to store the results
count_true_positives = {}
used = set()
for class_index, class_name in enumerate(classes.keys()):
    count_true_positives[class_name] = 0
    """
     Load predictions of that class
    """
#     predictions_file = TEMP_FILES_PATH + "/" + class_name + "_predictions.json"
    predictions_data = classes[class_name][0]#json.load(open(predictions_file))
    ground_truth_data = classes[class_name][1]
    """
     Assign predictions to ground truth objects
    """
    nd = len(predictions_data)
    tp = [0] * nd # creates an array of zeros of size nd
    fp = [0] * nd
    for idx, prediction in enumerate(predictions_data):
        ovmax = -1
        gt_match = -1
        bb = prediction.getPos()
        for obj in ground_truth_data:
            bbgt = obj.getPos()
            ov = IOU(bb, bbgt)
            if ov > ovmax:
                ovmax = ov
                gt_match = obj
        min_overlap = MINOVERLAP
        
        if ovmax >= min_overlap and gt_match not in used:
            # true positive
            tp[idx] = 1
            used.add(gt_match)
            count_true_positives[class_name] += 1
        else:
            # false positive
            fp[idx] = 1
        
    #print(tp)
    # compute precision/recall
    cumsum = 0
    for idx, val in enumerate(fp):
        fp[idx] += cumsum
        cumsum += val
    cumsum = 0
    for idx, val in enumerate(tp):
        tp[idx] += cumsum
        cumsum += val
    #print(tp)
    rec = tp[:]
    for idx, val in enumerate(tp):
        rec[idx] = float(tp[idx]) / gt_counter_per_class[class_name]
    #print(rec)
    prec = tp[:]
    for idx, val in enumerate(tp):
        prec[idx] = float(tp[idx]) / (fp[idx] + tp[idx])
    #print(prec)

    ap, mrec, mprec = voc_ap(rec[:], prec[:])
    sum_AP += ap

    ap_dictionary[class_name] = ap

    n_images = counter_images_per_class[class_name]
    lamr, mr, fppi = log_average_miss_rate(np.array(rec), np.array(fp), n_images)
    lamr_dictionary[class_name] = lamr


# results_file.write("\n# mAP of all classes\n")
# mAP = sum_AP / n_classes
# text = "mAP = {0:.2f}%".format(mAP*100)
# results_file.write(text + "\n")
# print(text)


NameError: name 'gt_counter_per_class' is not defined

[2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]
